In [1]:
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices('GPU'))
print("Built with CUDA:", tf.test.is_built_with_cuda())

2025-09-05 11:56:16.006872: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-05 11:56:16.043560: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-05 11:56:16.043602: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-05 11:56:16.044967: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-05 11:56:16.052009: I tensorflow/core/platform/cpu_feature_guar

GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Built with CUDA: True


In [2]:
import os
import numpy as np
import tensorflow as tf
from data_loader import data_generator
from sklearn.metrics import f1_score, precision_score, recall_score
import datetime
from helpers import f1 as f1_metric
from model import lstm_vit

def set_random_seeds(seed):
    """Set random seeds for reproducibility."""
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def create_data_generators():
    """Create data generators for training and validation."""
    train_generator = data_generator(
        image_dir='data_v3_processed/train/images/images',
        mask_dir='data_v3_processed/train/masks/masks',
        batch_size=8
    )
    
    val_generator = data_generator(
        image_dir='data_v3_processed/val/images/images',
        mask_dir='data_v3_processed/val/masks/masks',
        batch_size=8,
        train=False,
        sequence_length=10
    )

    return train_generator, val_generator

def train_and_evaluate_model(run_number, train_generator, val_generator, 
                           epochs=7, steps_per_epoch=1500, validation_steps=5, 
                           random_seed=None):
    """Train a single model and return evaluation metrics."""
    print(f"\nTraining Run {run_number + 1}/5 (Seed: {random_seed})")
    print("=" * 50)
    
    if random_seed is not None:
        set_random_seeds(random_seed)
    
    model = lstm_vit()
    
    model.compile(optimizer='adam', 
                 loss='binary_crossentropy', 
                 metrics=[f1_metric])
    
    try:
        history = model.fit(train_generator,
                           validation_data=val_generator,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_steps=validation_steps,
                           verbose=1)

        model_path = os.path.join("models", f"lstm_vit_run2.h5")
        os.makedirs("models", exist_ok=True)
        model.save(model_path)
        
        return model_path
        
    except Exception as e:
        print(f"ERROR in training: {e}")
        return None

# Main execution
random_seeds = [123]

for run in range(1):
    # CREATE FRESH GENERATORS FOR EACH RUN
    train_generator, val_generator = create_data_generators()
    
    path = train_and_evaluate_model(
        run, train_generator, val_generator, random_seed=random_seeds[run]
    )
    tf.keras.backend.clear_session()


Training Run 1/5 (Seed: 123)


2025-09-05 11:56:22.913028: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2025-09-05 11:56:22.913389: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46672 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:21:00.0, compute capability: 8.9


Building sequence list and filtering empty sequences...
Total valid sequences: 220
Skipped empty sequences: 270
Epoch 1/7


2025-09-05 11:56:35.543967: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inLSTM_ViT_ConvLSTM/bidirectional/forward_conv_lstm2d/while/body/_1/LSTM_ViT_ConvLSTM/bidirectional/forward_conv_lstm2d/while/dropout_7/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2025-09-05 11:56:36.776210: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2025-09-05 11:56:40.675379: I external/local_xla/xla/service/service.cc:168] XLA service 0x7efc044aa9f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-09-05 11:56:40.675417: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2025-09-05 11:56:40.681017: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRO

1500/1500 [==============================] - ETA: 0s - loss: 0.0275 - f1: 7.0573e-06Building sequence list and filtering empty sequences...
Total valid sequences: 5
Skipped empty sequences: 0
1500/1500 [==============================] - 786s 506ms/step - loss: 0.0275 - f1: 7.0573e-06 - val_loss: 0.0274 - val_f1: 0.0000e+00
Epoch 2/7
1500/1500 [==============================] - 749s 500ms/step - loss: 0.0056 - f1: 0.6384 - val_loss: 0.0097 - val_f1: 0.8036
Epoch 3/7
1500/1500 [==============================] - 748s 499ms/step - loss: 0.0029 - f1: 0.8431 - val_loss: 0.0123 - val_f1: 0.7199
Epoch 4/7
1500/1500 [==============================] - 747s 498ms/step - loss: 0.0024 - f1: 0.8659 - val_loss: 0.0097 - val_f1: 0.8273
Epoch 5/7
1500/1500 [==============================] - 746s 497ms/step - loss: 0.0021 - f1: 0.8826 - val_loss: 0.0114 - val_f1: 0.8229
Epoch 6/7
1500/1500 [==============================] - 746s 497ms/step - loss: 0.0020 - f1: 0.8878 - val_loss: 0.0123 - val_f1: 0.8232


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
